# Modular

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import time
from nltk.corpus import wordnet
import re
import enchant
import random
import numpy as np

# Read data

In [3]:
filename = '/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/data/testval_data.csv'
reviews = pd.read_csv(filename)

In [4]:
reviews.shape

(1016664, 8)

# Test number

In [5]:
N = 100000
NN = reviews.shape[0]

In [6]:
np.random.seed(615)
index_ori = np.random.randint(0, NN, N)

In [7]:
index_ori

array([965810, 935900, 687107, ..., 936467, 548678, 965742])

In [8]:
yelp = reviews.copy()
yelp_ori = yelp.copy()

In [10]:
yelp = reviews.iloc[index_ori,:].copy()
yelp_ori = yelp.copy()

In [9]:
yelp_ori.shape

(1016664, 8)

# 1 加上长度变量

In [5]:
yelp.loc[:, 'text length'] = yelp['text'].apply(len)

# 2 改缩写和换行符

In [10]:
yelp.iloc[335, 2]

"This is the worst Sushi restaurant. I guess I never learn because they gave me food poisoning more than once from the eel.\n\nDon't get adventurous here because they'll make you hate all the good stuff that you would enjoy otherwise. One such example is their uni which is not fresh and served in too large a portion.\n\nThe service is also terrible. They have a lame attitude and last time I got the caterpillar roll, I told the waiter that it tasted bad so he laughed and walked away not offering to exchange it. I ended up paying for it with part of his tip.\n\nTheir rolls are humongous so if you're a glutton who doesn't care about taste, this could be your sushi place. Most rolls contain lots of fake crab."

In [11]:
shortcut_dir = {'n\'t': ' not', '\'s': ' is', '\'ve': ' have', '\'d': ' would', '\'m': ' am',
                'Ive': 'I have', 'cannot': 'can not', '\\n': ' '}

In [12]:
print('\\n')

\n


In [13]:
t1 = time.time()
def shortcut_remove(review):
    review = review.lower()
    for i in shortcut_dir:
        review = re.sub(i, shortcut_dir[i], review)
    return review
yelp2 = yelp.copy()
yelp2['text'] = yelp2['text'].apply(shortcut_remove)
time.time() - t1

14.250269889831543

In [14]:
yelp2.shape

(1016664, 8)

In [17]:
# i=2580
# print(yelp['text'][i])
# print(yelp2['text'][i])

In [18]:
# yelp2['text'][3]

# 3 Delete unEnglish

In [16]:
# d = enchant.Dict("en_US")
    
# def unEnglish_remove(review):
#     review = re.sub('[^a-zA-Z]',' ', review)
#     words = pd.Series(review.split())
#     index = words.apply(d.check)
#     if(len(index)==0):
#         return True
#     else:
#         return(sum(index)/len(index)<0.8)
# t1 = time.time()
# index = yelp2['text'].apply(unEnglish_remove)
# time.time() - t1

2685.2600350379944

In [30]:
# xxxx = pd.DataFrame(list(np.where(index)))
# filename_shi = "/Users/Lyf/Desktop/test_unen.csv"
# xxxx.to_csv(filename_shi, index = False)

In [31]:
# xxxx.shape

(1, 3291)

In [32]:
# yelp3 = yelp2[-index].copy()

In [26]:
# filename_shi = "/Users/Lyf/Desktop/train_unen10w.csv"
# yelp3.to_csv(filename_shi, index = False)

In [2]:
# i=0
# yelp2['text'][index][(i*10):(i*10+9)]

In [3]:
# yelp2['text'][35]

In [4]:
# yelp2['text'][[24,1167,2580,3750]]

# Get positive and negative

In [5]:
# reviews_total = ''
# for i in range(N):
#     reviews_total = reviews_total+" "+yelp.loc[i, 'text']

In [6]:
# reviews_total
# positive_words = ["nice","prime","great","awesome","yummy","friendly","delicious",
#                   "love","spicy","good","favorite","well","clean","enjoy"]
# negetive_words = ["suck","terrible","amazing","messy","worse","hit","miss"]

In [7]:
# print(reviews['text'][9])

# 4 Add not

In [33]:
yelp3 = yelp2.copy()

In [34]:
inverter = ['not', 'never', 'lack', 'no']
def add_not(sentence):
    words = sentence.split().copy()
    not_index = 0
    for index, word in enumerate(words):
        if(not_index != 0):
            words[index] = 'not' + words[index]
        if(re.match('.*[,.?!\'\"]$', word)):
            not_index = 0
        if(word in inverter):
            not_index = 1
    #         print(index)
    sentence = " ".join(words)
    return(sentence)
t1 = time.time()
yelp4 = yelp3.copy()
yelp4['text'] = yelp3['text'].apply(add_not)
time.time() - t1

142.88873982429504

In [15]:
# i=0
# print(yelp3['text'][i])
# print(yelp4['text'][i])

# 5 去标点

In [8]:
# yelp4 = yelp2.copy()

In [9]:
# yelp4.iloc[16824,2]

In [10]:
# def punctuation_remove(review):
#     review = review.lower()
#     review = re.sub("[^a-zA-Z]", " ", review)
#     return review
# t1 = time.time()
# yelp5 = yelp4.copy()
# yelp5['text'] = yelp4['text'].apply(punctuation_remove)
# time.time() - t1

In [11]:
# filename_chen = "../data/chen_first"+str(N)+".csv"
# yelp4.to_csv(filename_chen, index = False)

In [12]:
# yelp5.shape

In [13]:
# filename_shi = "/Users/Lyf/Desktop/test_cnn.csv"
# yelp5.to_csv(filename_shi, index = False)

# 6 去不重要的词

In [14]:
# feature = pd.read_csv("../../data/feature3_all.csv")

In [15]:
# myfeature = list(feature["word"])

In [16]:
# myfeature

In [17]:
# def unim_remove(review):
#     review = review.split()
#     useful_review = [w for w in review if w in myfeature]  
#     return " ".join([w for w in useful_review])
# t1 = time.time()
# yelp6 = yelp5.copy()
# yelp6['text'] = yelp5['text'].apply(unim_remove)
# time.time() - t1

In [ ]:
# filename_shi = "/Users/Lyf/Desktop/train_cnn10w.csv"
# yelp6.to_csv(filename_shi, index = False)

# 5 去时态

In [20]:
# from nltk.stem.wordnet import WordNetLemmatizer 
# lem = WordNetLemmatizer()

# from nltk.stem.porter import PorterStemmer 
# stem = PorterStemmer()
# def tense_remove(review):
#     review = review.split()
#     return " ".join([stem.stem(w) for w in review])
# t1 = time.time()
# yelp5 = yelp4.copy()
# yelp5['text'] = yelp5['text'].apply(tense_remove)
# time.time() - t1

# 7 去停词

In [21]:
# stops = set(stopwords.words("english"))
# stops2 = list(stops)
# stops22 = ['not_'+i for i in stops2]
# stop_all = stops2+stops22

In [18]:
# def noise_remove(review):
#     review = review.split()
#     useful_review = [w for w in review if not w in stop_all]  
#     return " ".join([w for w in useful_review])
# t1 = time.time()
# yelp7 = yelp6.copy()
# yelp7['text'] = yelp7['text'].apply(noise_remove)
# time.time() - t1

In [51]:
filename_li = "/Users/Lyf/Desktop/cnn10w.csv"

In [52]:
# filename_li = "/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/test2.R"

In [53]:
filename_li

'/Users/Lyf/Desktop/cnn10w.csv'

In [54]:
yelp5.to_csv(filename_li, index = False)

In [16]:
filename_shi = "../data/shi_first"+str(N)+"withoutNot.csv"
yelp7.to_csv(filename_shi, index = False)

In [20]:
filename_li = "../data/li_first"+str(N)+"_2.csv"
yelp6.to_csv(filename_li, index = False)

In [27]:
yelp5

,Id,name,text,date,city,longitude,latitude,categories
965810,965811,Blanco Tacos + Tequila,food was just ok bar area was dirty very ni...,2015-09-21,Phoenix,-112.028389,33.510456,"['Vegetarian', 'Mexican', 'Restaurants']"
935900,935901,La Cage Aux Sports,this is actually a very nice restaurant inside...,2015-05-10,Montréal,-73.552638,45.506790,"['Sandwiches', 'Burgers', 'Restaurants', 'Chic..."
687107,687108,Rebel Republic,great escape from the strip for sports great...,2015-04-20,Las Vegas,-115.188286,36.145743,"['Burgers', 'American (New)', 'Sports Bars', '..."
153400,153401,Hakkasan Nightclub,do not bother with guestlist if you re trying ...,2014-09-30,Las Vegas,-115.168109,36.103799,"['Chinese', 'Nightlife', 'Restaurants', 'Dance..."
289695,289696,Imadake,i love this place the ramen noodles are great...,2011-03-03,Montréal,-73.584954,45.488794,"['Japanese', 'Pubs', 'French', 'Restaurants', ..."
185596,185597,N9NE Steakhouse,my wife and i asked around for a nice steak ho...,2009-11-15,Las Vegas,-115.194100,36.114750,"['Steakhouses', 'Restaurants', 'American (Trad..."
968125,968126,Cafe Rio,worst cafe rio i have ever been to consistent...,2014-05-09,Las Vegas,-115.208013,36.279467,"['Restaurants', 'Mexican']"
224448,224449,Original Pancake House,what a disappointment nothing special about ...,2012-08-06,Scottsdale,-111.932999,33.502593,"['Breakfast & Brunch', 'Restaurants', 'Food', ..."
983577,983578,Chop Chop Leith,went to the one in leith was not bad at all ...,2012-04-19,Edinburgh,-3.172733,55.977025,"['Restaurants', 'Chinese']"
551258,551259,Quiznos,great sandwich and friendly staff i live in s...,2016-03-17,Phoenix,-112.030081,33.679037,"['Fast Food', 'Restaurants', 'Sandwiches', 'De..."
